# Assignment 8: Correlation Analysis and Hypothesis Testing

## Objective

The field of statistics is divided into two major parts: descriptive and inferential. In this assignment, we will cover two important topics in statistics: correlation analysis and hypothesis testing, where the former belongs to the descriptive part and the latter belongs to the inferential part. After this assignment, you will be able to answer the following questions:

**Correlation Analysis**
1. How to visualize the relationship between two variables?  
2. What is Pearson's correlation? How to compute it?
3. What is Spearman's rank correlation? How to compute it?
4. What's the difference between Pearson's correlation and Spearman's rank correlation? 


**Hypothesis Testing**
1. Why A/B testing?  
2. What is a permutation test? How to implement it?
3. What is p-value? How to avoid p-hacking? 
4. What is a chi-squared test? How to implement it?


In this assignment, you can use [pandas](https://pandas.pydata.org/) or PySpark to manipulate data, and use [matplotlib](https://matplotlib.org/) or [seaborn](seaborn.pydata.org) to make plots. 

## Part 1. Correlation Analysis

As a data scientist, you often face this kind of question: "Are A and B correlated?" For example, 

* Do Canadian Currency and Oil Price move together?
* Do Vancouver Housing Price and US Stock Market have any correlation?
* Are GPA and Gender independent? 

To answer these questions, you need to conduct correlation analysis. 

Imagine you are a data scientist working at a real-estate company. You download a dataset from [property_tax_report_2018.zip](property_tax_report_2018.zip). The dataset contains information on properties from BC Assessment (BCA) and City sources in 2018.  You can find the schema information of the dataset from this [webpage](http://data.vancouver.ca/datacatalogue/propertyTaxAttributes.htm). 

You may think that for a newly built house, it tends to have a higher price than the ones built decades ago. In this assignment, your first job is to figure out whether YEAR_BUILT and HOUSE_PRICE are correlated. 

We first load the data as a DataFrame. 

In [1]:
import pandas as pd

df = pd.read_csv("property_tax_report_2018.csv")

df['HOUSE_PRICE'] = df.apply(lambda x: (x['CURRENT_LAND_VALUE']+x['CURRENT_IMPROVEMENT_VALUE'])/1000000.0, axis = 1)

df.head()

,PID,LEGAL_TYPE,FOLIO,LAND_COORDINATE,ZONE_NAME,ZONE_CATEGORY,LOT,BLOCK,PLAN,DISTRICT_LOT,...,CURRENT_LAND_VALUE,CURRENT_IMPROVEMENT_VALUE,TAX_ASSESSMENT_YEAR,PREVIOUS_LAND_VALUE,PREVIOUS_IMPROVEMENT_VALUE,YEAR_BUILT,BIG_IMPROVEMENT_YEAR,TAX_LEVY,NEIGHBOURHOOD_CODE,HOUSE_PRICE
0,025-734-601,STRATA,7.500400e+11,75004024,C-2,Commercial,25,NaN,BCS498,2027,...,592000,242000,2018,472000.0,238000.0,2003.0,2003.0,NaN,3,0.834
1,029-700-868,STRATA,6.381830e+11,63818250,CD-1 (464),Comprehensive Development,132,NaN,EPS2983,200A,...,715000,327000,2018,603000.0,329000.0,NaN,NaN,NaN,13,1.042
2,029-814-227,STRATA,1.708260e+11,17082596,CD-1 (535),Comprehensive Development,25,NaN,EPS3173,311,...,507000,273000,2018,416000.0,273000.0,NaN,NaN,NaN,12,0.780
3,029-918-731,STRATA,6.401940e+11,64019406,IC-3,Light Industrial,40,26,EPS2425,200A,...,227000,170000,2018,168000.0,170000.0,NaN,NaN,NaN,13,0.397
4,017-393-400,STRATA,6.011150e+11,60111496,CD-1 (233),Comprehensive Development,7,NaN,LMS75,185,...,801000,380000,2018,531000.0,385000.0,1991.0,1991.0,NaN,27,1.181


### Task A. Visualizations

Since the housing price varies a lot by locations, we will only consider the houses whose <font color='blue'>postcode starts with 'V6A'</font>. Furthermore, we remove the houses that were <font color='blue'>built before 1900</font>.

<img src="img/v6a.png", width=500/>

In the following, please make two subplots in one row. For the left subplot, it is a scatter plot with X = `YEAR_BUILT` and Y = `HOUSE_PRICE`; for the right subplot, it is a hexbin plot (gridsize = 20) with X = `YEAR_BUILT` and Y = `HOUSE_PRICE`.

In [2]:
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

# Use custom Hexbin function- No direct API support in plotly.
def compute_hexbin(x, y, gridsize=100, bins=None, cmap=plt.cm.Blues):
    collection = plt.hexbin(x, y, bins=bins, gridsize=gridsize)
    plt.close()

    pts_in_hexagon = collection.get_array()

    #compute colors for the svg shapes
    colors = ["#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b, _ in 220*cmap(Normalize()(pts_in_hexagon))]
    length = len(pts_in_hexagon)
    for iter_val in range(length):
        if int(pts_in_hexagon[iter_val]) == 0:
            colors[iter_val] = "#%02x%02x%02x" % (int(255), int(255), int(255))
        
    # coordinates for single hexagonal patch
    hx = [0, .5, .5, 0, -.5, -.5]
    hy = [-.5/np.cos(np.pi/6), -.5*np.tan(np.pi/6), .5*np.tan(np.pi/6),
          .5/np.cos(np.pi/6), .5*np.tan(np.pi/6), -.5*np.tan(np.pi/6)]

    # number of hexagons needed
    m = len(collection.get_offsets())

    # scale of hexagons
    n = (x.max() - x.min()) / gridsize

    # y_scale to adjust for aspect ratio
    y_scale = (y.max() - y.min())/(x.max() - x.min())

    # coordinates for all hexagonal patches
    hxs = np.array([hx]*m)*n + np.vstack(collection.get_offsets()[:,0])
    hys = np.array([hy]*m)*n*y_scale + np.vstack(collection.get_offsets()[:,1])

    return hxs.tolist(), hys.tolist(), colors, pts_in_hexagon

# Filtering out rows that have 'YEAR_BUILT' < 1900 and 'PROPERTY_POSTAL_CODE' other ones starting with 'V6A'.
df1 = df[df['YEAR_BUILT']>=1900]
df1.dropna(subset = ['PROPERTY_POSTAL_CODE'],inplace=True)
df1 = df1[df1['PROPERTY_POSTAL_CODE'].str.startswith('V6A')]
df1.head()

# Create plot data for Scatter plot.
trace1 = go.Scatter(text = '(Year, House Price)',
                x = df1['YEAR_BUILT'],
                y = df1['HOUSE_PRICE'],
                name = 'Years built vs property price',
                mode = 'markers',
                marker = dict(
                    size = 10,
                    color = 'maroon',
                    line = dict(
                        width = 1.5,
                        color = 'pink'
                        )
                    ),
                opacity = 0.7
                )

# Create plot data for Hexbin plot.
x, y, color_list, pts_in_hexagon = compute_hexbin(df1['YEAR_BUILT'], df1['HOUSE_PRICE'], gridsize=30)

shape_container = []
hover_point_x = []
hover_point_y = []

for x_list, y_list, color in zip(x, y, color_list):

    #Create the svg path based on the computed points
    svg_path = 'M {},{} L {},{} L {},{} L {},{} L{},{} L{},{}'\
        .format(x_list[0], y_list[0],
                x_list[1], y_list[1],
                x_list[2], y_list[2],
                x_list[3], y_list[3],
                x_list[4], y_list[4],
                x_list[4], y_list[1])

    #Create hover point from the hexagon, witch is the center of gravity
    hover_point_x.append(round((max(x_list) - min(x_list))/2+min(x_list), 2))
    hover_point_y.append(round((max(y_list) - min(y_list))/2+min(y_list), 2))

    shape_container.append({
          "fillcolor": color,
          "line": {
            "color": color,
            "width": 2
          },
          "path": svg_path,
          "type": "path"
        })

trace0 = go.Scattergl(x=hover_point_x,
                   y=hover_point_y,
                   mode='markers',
                   text='(Year, House price)',
                   name = 'Years built vs property price',
                   )

trace0['marker']['reversescale'] = True
trace0['marker']['colorscale'] = 'Blues'
trace0['marker']['color'] = pts_in_hexagon
trace0['marker']['size'] = 0
trace0['opacity'] = 0.7
trace0['text'] = list(map(lambda z: 'Amount of points: {}'.format(int(z)), pts_in_hexagon))


# Create layout for the plot
layout = {'shapes':shape_container,
          'width': 850,
          'height': 700,
          'title': 'Year built vs House price plots.',
          'hovermode':'closest',
        }

# Create subplots
fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('Hexbin Plot', 'Scatter Plot'))

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)

fig['layout']['xaxis1'].update(title='Year Built', showgrid=False, type='linear')
fig['layout']['xaxis2'].update(title='Year Built', showgrid=False, type='linear')

fig['layout']['yaxis1'].update(title='price of the property (in million)', showgrid=False, type='linear')
fig['layout']['yaxis2'].update(title='price of the property (in million)', showgrid=False, type='linear')

# Show the plot.
init_notebook_mode(connected=True)
fig['layout'].update(layout, height=650, width=1500)
iplot(fig, filename='Q1-Scatter-Hexbin')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



Please write down the **two** most interesting findings that you draw from the plot.

**Findings**
1. Majority of the property in the dataset, built after (or in 1900) 1900 and having postal code starting with 'V6A' have their prices below 20 million. Another interesting thing about this is that, the spread is almost even and dense through-out the period under consideration.
2. From the hexbin plot we get to see that around the year 1998 has seen '567' points lying in its hexagonal bin/region closely followed by year 2014, which has '533' points lying in its heaxagonal bin/region- this shows the density of points correlated to that partciular time of the year.

The above plots provide a general impression of the relationship between variables. There are some other visualizations that can provide more insight. One option is to bin one variable and plot percentiles of the other. 


In the following, please make three subplots in a row, where each subplot is a scatter plot with X = YEAR_BUILT and Y = HOUSE_PRICE. 
* The first subplot shows how the 25th percentile of `HOUSE_PRICE` changes over years (X = `YEAR_BUILT`, Y = `25TH_HOUSE_PRICE`); 
* The second subplot shows how the 50th percentile of `HOUSE_PRICE` changes over years (X = `YEAR_BUILT`, Y = `50TH_HOUSE_PRICE`); 
* The third subplot shows how the 75th percentile of `HOUSE_PRICE` changes over years (X = `YEAR_BUILT`, Y = `75TH_HOUSE_PRICE`);  


In [3]:
# Prepare data for plotting.
twentyfifth_percentile_df = df1.quantile(0.25)
fiftyth_percentile_df = df1.quantile(0.50) # aka the median
median = df1.median()
seventyfifth_percentile_df = df1.quantile(0.75)
print('Max value of House price for DF:', df1['HOUSE_PRICE'].max())

df_twentyfifth = df1.groupby('YEAR_BUILT')['HOUSE_PRICE'].quantile(0.25)
df_fiftyth = df1.groupby('YEAR_BUILT')['HOUSE_PRICE'].quantile(0.5)
df_seventyfifth = df1.groupby('YEAR_BUILT')['HOUSE_PRICE'].quantile(0.75)

# Create plot data for Scatter plot.
# Feed 25%le house price vs year built plot data.
trace0 = go.Scatter(text = '25%le value',
                x = df_twentyfifth.index,
                y = df_twentyfifth.values,
                name = 'Years built vs 25th percentile property price',
                mode = 'markers',
                marker = dict(
                    size = 10,
                    color = 'lightgreen',
                    line = dict(
                        width = 1.5,
                        color = 'blue'
                        )
                    ),
                opacity = 0.7
                )

# Feed 50%le house price vs year built plot data.
trace1 = go.Scatter(text = '50%le value',
                x = df_fiftyth.index,
                y = df_fiftyth.values,
                name = 'Years built vs 50th percentile property price',
                mode = 'markers',
                marker = dict(
                    size = 10,
                    color = 'purple',
                    line = dict(
                        width = 1.5,
                        color = 'black'
                        )
                    ),
                opacity = 0.7
                )

# Feed 75%le house price vs year built plot data.
trace2 = go.Scatter(text = '75%le value',
                x = df_seventyfifth.index,
                y = df_seventyfifth.values,
                name = 'Years built vs 75th percentile property price',
                mode = 'markers',
                marker = dict(
                    size = 10,
                    color = 'black',
                    line = dict(
                        width = 1.5,
                        color = 'cyan'
                        )
                    ),
                opacity = 0.7
                )

# Create layout for the plot
layout = go.Layout(
        title="Year Built VS Different percentile house prices.",
        xaxis=dict(title='Year Built',
            ),
        yaxis=dict(title='price (in million)',
            ),
        showlegend=True,
        barmode='overlay',
        legend=dict(bgcolor='lightgray',
            bordercolor='gray',
            borderwidth=2
        )
    )

# Create subplots
fig = tools.make_subplots(rows=1, cols=3, subplot_titles=('25%le house price change with year built.', '50%le house price change with year built', '75%le house price change with year built'))

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 1, 3)

fig['layout']['xaxis1'].update(title='Year Built', showgrid=False, type='linear')
fig['layout']['xaxis2'].update(title='Year Built', showgrid=False, type='linear')
fig['layout']['xaxis3'].update(title='Year Built', showgrid=False, type='linear')

fig['layout']['yaxis1'].update(title='25th percentile price (in million)', showgrid=True, type='linear')
fig['layout']['yaxis2'].update(title='50th percentile price (in million)', showgrid=True, type='linear')
fig['layout']['yaxis3'].update(title='75th percentile price (in million)', showgrid=True, type='linear')

# Show the plot.
init_notebook_mode(connected=True)
fig['layout'].update(layout, height=500, width=2000)
iplot(fig, filename='Q2-percentile-scatterplots')

Max value of House price for DF: 162.672
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]



Please write down the **two** most interesting findings that you draw from the plot.

**Findings**
1. Year 1943 has the highest 25%le, 50%le and 75%le values for house price. But, this does not hold good for other years under consideration as their 25%le rank, 50%le and 75%le values rank differently.
2. Year 1952, has the second highest 75%le house price value of 43.93 million. However, its 25%le and 50%le house price values are on the lower side- 3.41 million and 4.32 million respectively. This clearly depicts that a lot of high valued property got built in this year.

### Task B. Correlation Coefficient

A correlation coefficient is a numerical measure of some type of correlation, meaning a statistical relationship between a pair of variables. 

In the following, please compute the Pearson's correlation as well as Spearman's rank correlation for three pairs of variables: <`25TH_HOUSE_PRICE`, `YEAR_BUILT`>, <`50TH_HOUSE_PRICE`, `YEAR_BUILT`>, and <`75TH_HOUSE_PRICE`, `YEAR_BUILT`>, and then print out your results. 

Note that it is OK to use this [function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) to do this task, but make sure that you understand how the numbers are computed. 

In [4]:
# create three different dataframes for 25%le, 50%le and 75%le house price values with year as the other column.
twentyfifth = pd.DataFrame({'YEAR_BUILT':df_twentyfifth.index, '25TH_HOUSE_PRICE':df_twentyfifth.values})
twentyfifth.head()
fiftyth = pd.DataFrame({'YEAR_BUILT':df_fiftyth.index, '50TH_HOUSE_PRICE':df_fiftyth.values})
seventyfifth = pd.DataFrame({'YEAR_BUILT':df_seventyfifth.index, '75TH_HOUSE_PRICE':df_seventyfifth.values})

# compute pearson's cofficient for all three pairs mentioned.
print('Pearson\'s correlation for the pair <25TH_HOUSE_PRICE, YEAR_BUILT>:', twentyfifth['25TH_HOUSE_PRICE'].corr(twentyfifth['YEAR_BUILT'], method='pearson'), "\n")
print('Pearson\'s correlation for the pair <50TH_HOUSE_PRICE, YEAR_BUILT>:', fiftyth['50TH_HOUSE_PRICE'].corr(twentyfifth['YEAR_BUILT'], method='pearson'), "\n") 
print('Pearson\'s correlation for the pair <75TH_HOUSE_PRICE, YEAR_BUILT>:', seventyfifth['75TH_HOUSE_PRICE'].corr(twentyfifth['YEAR_BUILT'], method='pearson'), "\n")

# compute spearman's rank cofficient for all three pairs mentioned.
print('Spearman\'s Rank correlation for the pair <25TH_HOUSE_PRICE, YEAR_BUILT>:', twentyfifth['25TH_HOUSE_PRICE'].corr(twentyfifth['YEAR_BUILT'], method='spearman'), "\n")
print('Spearman\'s Rank correlation for the pair <50TH_HOUSE_PRICE, YEAR_BUILT>:', fiftyth['50TH_HOUSE_PRICE'].corr(twentyfifth['YEAR_BUILT'], method='spearman'), "\n") 
print('Spearman\'s Rank correlation for the pair <75TH_HOUSE_PRICE, YEAR_BUILT>:', seventyfifth['75TH_HOUSE_PRICE'].corr(twentyfifth['YEAR_BUILT'], method='spearman'), "\n") 

Pearson's correlation for the pair <25TH_HOUSE_PRICE, YEAR_BUILT>: -0.24453312810700037 

Pearson's correlation for the pair <50TH_HOUSE_PRICE, YEAR_BUILT>: -0.2375040142180012 

Pearson's correlation for the pair <75TH_HOUSE_PRICE, YEAR_BUILT>: -0.21267690503199593 

Spearman's Rank correlation for the pair <25TH_HOUSE_PRICE, YEAR_BUILT>: -0.5721119349437934 

Spearman's Rank correlation for the pair <50TH_HOUSE_PRICE, YEAR_BUILT>: -0.5098468832130557 

Spearman's Rank correlation for the pair <75TH_HOUSE_PRICE, YEAR_BUILT>: -0.4004424778761062 



Please write down the **two** most interesting findings that you draw from the result. 

**Findings**
1. Both spearman and pearson correlation on the whole show the same trend (both negatively correlated and have the same ordering) but it should be noted that Spearman Rank correlation seems to penalize the values higher(this is because it takes into account the non-linearity in the data unlike pearson correlation, which simply considers only linear relationships) than its counterpart (here)- Pearson.
2. In both cases we can see that, the pair:<25TH_HOUSE_PRICE, YEAR_BUILT> has the highest value, from which we can infer that 25%le house prices are better correlated with respective year_built values over 50%le house prices and 75%le house price values. 

## Part 2. Hypothesis Testing

In many situations, we cannot get the full population but only a sample. If we derive an interesting result from a sample, how likely can we derive the same result from the entire population? In other words, we want to know whether this result is a true finding or it just happens in the sample by chance. Hypothesis testing aims to answer this fundamental question. 

### Task C. A/B Testing
> Acknowledgment: Thank [Greg Baker](http://www.cs.sfu.ca/~ggbaker/) for helping me to prepare this task.

A very common technique to evaluate changes in a user interface is A/B testing: show some users interface A, some interface B, and then look to see if one performs better than the other.

Suppose I started an A/B test on CourSys. Here are the two interfaces that I want to compare with. I want to know whether a good placeholder in the search box can attract more users to use the `search` feature.

<img src="img/ab-testing.png", width=800/>

The provided [searchlog.json](./searchlog.json) has information about users' usage. The question I was interested in: is the number of searches per user different?

To answer this question, we need to first pick up a **test statistic** to quantify how good an interface is. Here, we choose "the search_count mean". 

Please write the code to compute **the difference of the search_count means between interface A and Interface B.** 

In [5]:
# Read Data.
df_data = pd.read_json('searchlog.json', lines=True)
df_data.head()

,is_instructor,search_count,search_ui,uid
0,True,2,A,6061521
1,False,0,A,11986457
2,False,0,A,15995765
3,True,0,B,9106912
4,False,0,A,9882383


In [6]:
# Create dataframe for 'A' and 'B' interface values seperately.
dfA = df_data[df_data['search_ui']=='A']
dfB = df_data[df_data['search_ui']=='B']

# Compute mean on A and B dfs based on 'search_count'.
search_count_meanA = dfA['search_count'].mean()
search_count_meanB = dfB['search_count'].mean()

# calculate the difference of means.
diff_search_count_means = search_count_meanA - search_count_meanB
print('A-B difference:', diff_search_count_means)
print('Absolute difference of means:', abs(diff_search_count_means))

A-B difference: -0.13500569535052287
Absolute difference of means: 0.13500569535052287


Suppose we find that the mean value increased by 0.135. Then, we wonder whether this result is just caused by random variation. 

We define the Null Hypothesis as
 * The difference in search_count mean between Interface A and Interface B is caused by random variation. 
 
Then the next job is to check whether we can reject the null hypothesis or not. If it does, we can adopt the alternative explanation:
 * The difference in search_count mean  between Interface A and Interface B is caused by the design differences between the two.

We compute the p-value of the observed result. If p-value is low (e.g., <0.01), we can reject the null hypothesis, and adopt  the alternative explanation.  

Please implement a permutation test (numSamples = 10000) to compute the p-value. Note that you are NOT allowed to use an implementation in an existing library. You have to implement it by yourself.

In [7]:
# Write function here.
# perform permutation for one-tail test.
def permutation_test_onetail(data, lenA, lenB, diff, numsamples):
    counts = 0
    for iter_val in range(numsamples):
        np.random.shuffle(data)
        # considering one-tailed test.
        diff_count = data[:lenA].mean() - data[-lenB:].mean()
        if diff_count <= diff:
            counts += 1
    return_value = 1.0 - (float(counts) / float(numsamples))
    return return_value

# perform permutation test for two-tail test.
def permutation_test_twotail(data, lenA, lenB, diff, numsamples):
    counts = 0
    for iter_val in range(numsamples):
        np.random.shuffle(data)
        # considering two-tailed test. taking abs difference of means.
        diff_count = np.abs(data[:lenA].mean() - data[-lenB:].mean())
        if diff_count <= diff:
            counts += 1
    return_value = 1.0 - (float(counts) / float(numsamples))
    return return_value

# Call permutatation_test_do function with params and compute p-value.
numSamples = 10000
data = np.hstack([dfA['search_count'],dfB['search_count']])
diff = abs(diff_search_count_means)
lenA = len(dfA['search_count'])
lenB = len(dfB['search_count'])

# print(lenA, lenB)
p_value_singletail = permutation_test_onetail(data, lenA, lenB, diff, numSamples)
p_value_twotail = permutation_test_twotail(data, lenA, lenB, diff, numSamples)
print('p-value when considering one-tailed permutation test:', p_value_singletail, "\n")
print('p-value when considering two-tailed permutation test:', p_value_twotail, "\n")

if (p_value_singletail < 0.01) & (p_value_twotail < 0.01):
    print('Choose Alternative hyporthesis: The difference in search_count mean between Interface A and Interface B is caused by the design differences between the two.')
else:
    print('Choose null hypothesis: The difference in search_count mean between Interface A and Interface B is caused by random variation.')

p-value when considering one-tailed permutation test: 0.12939999999999996 

p-value when considering two-tailed permutation test: 0.2573 

Choose null hypothesis: The difference in search_count mean between Interface A and Interface B is caused by random variation.


Suppose we want to use the same dataset to do another A/B testing. We suspect that instructors are the ones who can get more useful information from the search feature, so perhaps non-instructors didn't touch the search feature because it was genuinely not relevant to them.

So we decide to repeat the above analysis looking only at instructors.

**Q. If using the same dataset to do this analysis, do you feel like we're p-hacking? If so, what can we do with it? **

**A.** I feel it could, if we are going to use the same data and on top of that if we are going to make an assumption that ' instructors are the ones who can get more useful information from the search feature, so perhaps non-instructors didn't touch the search feature'- then it could lead to a situation like p-hacking (where knowingly or unknowingly we manipualte the data inorder to get higher significant p-value). We can make sure that we frame the 'null-hypothesis' independently of the data under considertaion (that is do not be biased or use your knowledge from the previous result on this dataset to frame the new null hypothesis). Secondly, we can also decrease the confidence score (against which we check the p-value for accepting/rejecting the null-hypothesis) based on the number of new sub-classes we are splitting/categorizing the data into (eg. divide 0.05/numberofsubclasses). Also, we can make sure to take sufficient number of samples to get a good result. Lastly, keep the scope of the dataset broad enough to not fall into the pit of false positives and p-hacking (the dataset should have some diversity and variance so that it isn't biased).

### Task D. Chi-squared test 

There are tens of different hypothesis testing methods. It's impossible to cover all of them in one week. Given that this is an important topic in statistics, I highly recommend using your free time to learn some other popular ones such as <a href="https://en.wikipedia.org/wiki/Chi-squared_test">Chi-Squired test</a>, <a href="https://en.wikipedia.org/wiki/G-test">G-test</a>, <a href="https://en.wikipedia.org/wiki/Student%27s_t-test">T-test</a>, and <a href="https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test">Mann–Whitney U test</a>.

On the searchlog dataset, there are two categorical columns: `is_instructor` and `search_ui`. In Task D, your job is to first learn how a Chi-Squired test works by yourself and then use it to test whether `is_instructor` and `search_ui` are correlated. 

Please write code to compute the contingency table, the chi-squared stat, the degrees of freedom, and the p-value. Note that unlike Task C, you can call any function in an existing library. But, please make sure to understand how those functions are implemented. 

In [8]:
from scipy.stats import chi2_contingency

# Calculate contingency table.
count_isinsT_A = df_data[(df_data['is_instructor']==True) & (df_data['search_ui']=='A')]
count_isinsT_A = count_isinsT_A.count()[0]
count_isinsT_B = df_data[(df_data['is_instructor']==True) & (df_data['search_ui']=='B')]
count_isinsT_B = count_isinsT_B.count()[0]
count_isinsF_A = df_data[(df_data['is_instructor']==False) & (df_data['search_ui']=='A')]
count_isinsF_A = count_isinsF_A.count()[0]
count_isinsF_B = df_data[(df_data['is_instructor']==False) & (df_data['search_ui']=='B')]
count_isinsF_B = count_isinsF_B.count()[0]

print('True-A:', count_isinsT_A, 'True-B:', count_isinsT_B, 'False-A:', count_isinsF_A, 'False-B:', count_isinsF_B, "\n")

# show contingency table using crosstab feature and build the table using np.array.
data_crosstab = pd.crosstab(df_data['is_instructor'], df_data['search_ui'], margins = False) 
print('Contingency Table:\n', data_crosstab, "\n") 
contingency_table =  np.array([[count_isinsT_A, count_isinsT_B], [count_isinsF_A, count_isinsF_B]])   

# Calculate chi-squared stat, degrees of freedom and p-value.
chi_sqaured_stat, p_value, degree_of_freedom, expected = chi2_contingency(contingency_table)
print('Chi-sqaured stat:', chi_sqaured_stat, "\n")
print('p-value:', p_value, "\n")
print('degres of freedom:', degree_of_freedom, "\n")

True-A: 115 True-B: 120 False-A: 233 False-B: 213 

Contingency Table:
 search_ui        A    B
is_instructor          
False          233  213
True           115  120 

Chi-sqaured stat: 0.5473712356215868 

p-value: 0.45939379957424886 

degres of freedom: 1 



## Submission

Complete Tasks A-D in this [notebook](https://github.com/sfu-db/bigdata-cmpt733/blob/master/Assignments/A8/A8.ipynb), and submit it to the CourSys activity Assignment 8.